In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM5_ESM.xlsx', sheet_name='2G')

In [5]:
data_df

,ND,HFD,HFD+SAL,HFD+SANA
0,11920.0,15810.0,14645.0,8000.0
1,12350.0,14084.0,13665.0,12594.0
2,8800.0,15295.0,14220.0,13040.0
3,13305.0,9204.0,12525.0,9525.0
4,8480.0,15235.0,13685.0,9555.0
5,8910.0,21215.0,14645.0,8000.0
6,9490.0,16550.0,12430.0,16305.0
7,11945.0,15800.0,16720.0,8315.0
8,12350.0,11535.0,15710.0,8966.0
9,NaN,NaN,21075.0,11485.0


In [4]:
data_df.columns

Index(['ND', 'HFD', 'HFD+SAL', 'HFD+SANA'], dtype='object')

In [6]:
grp1, grp2, grp3, grp4 = data_df['ND'].dropna().values, \
data_df['HFD'].dropna().values, \
data_df['HFD+SAL'].dropna().values, \
data_df['HFD+SANA'].dropna().values

In [7]:
collect = [grp1, grp2, grp3, grp4]

In [8]:
def reorder_label(label):
    if label[-2:] == 'ND':
        return 'ND vs ' + label.split('vs')[0].strip()
    else:
        return label

In [9]:
for _ in [0]:

    print('N samples: ' + str(len(np.concatenate(collect))))
    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3, grp4)
    print('F(3, ' + str(len(np.concatenate(collect))-4) + ') = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate(collect), 
                                      groups=np.concatenate([['ND']*len(grp1), 
                                                             ['HFD']*len(grp2), 
                                                             ['HFD+SAL']*len(grp3), 
                                                             ['HFD+SANA']*len(grp4)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)
    bonf_df['Comparison'] = bonf_df['Comparison'].apply(reorder_label)

    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*collect).pvalue
    tukey_df = pd.DataFrame({'g1':['ND', 'ND', 'ND', 'HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD', 'HFD+SAL', 'HFD+SANA', 'HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[0][3], tukey[1][2], tukey[1][3], tukey[2][3]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())

N samples: 40
ANOVA:
F(3, 36) = 7.213320657843011, p = 0.000652509270860716
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |           1          |      0.988329   |
| HFD vs HFD+SANA     |                     |           0.0974806  |      0.0248623  |
| ND vs HFD           |                     |           0.0309115  |      0.0209937  |
| HFD+SAL vs HFD+SANA |                     |           0.0194079  |      0.00683533 |
| ND vs HFD+SAL       |                     |           0.00382032 |      0.00616317 |
| ND vs HFD+SANA      |                     |           1          |      0.996307   |


Here is Figure 2G:

![file](../img/pub_fig_2g.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38), we arrive at F(3, 36) = 7.21, p = 0.000653. However, the p-values shown for the post-hoc test are incorrect for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test) and are a closer match for Tukey's post-hoc test (with one rounding error in the ND vs HFD+SANA comparison). See summary table below.

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |           0.99          |           1.00          |      0.988   |
| HFD vs HFD+SANA     |               0.02      |           0.0975  |      0.0249  |
| ND vs HFD           |            0.02         |           0.0309  |      0.0210  |
| HFD+SAL vs HFD+SANA |             0.007        |           0.0194  |      0.00684 |
| ND vs HFD+SAL       |              Not shown       |           0.00382 |      0.00616 |
| ND vs HFD+SANA      |          0.997           |           1.00          |      0.996   |

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).